In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
import pickle

In [2]:
#Load Dataset
df=pd.read_csv("Churn_Modelling.csv")

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
print(df["Balance"].max())

250898.09


In [6]:
#Drop unnecessary columns
df.drop(columns=["RowNumber","CustomerId","Surname"],inplace=True)

In [7]:
df.sample(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
7848,433,France,Female,49,10,0.00,1,1,1,87711.61,0
9449,663,Germany,Male,28,8,123674.28,2,1,1,87985.20,0
1116,519,France,Male,42,8,0.00,2,1,1,101485.72,0
7671,405,France,Male,39,10,0.00,1,1,0,160810.85,1
3952,594,France,Female,34,7,141525.55,1,0,0,9443.15,0
6427,746,France,Male,37,2,0.00,2,1,0,143194.05,0
8040,624,Spain,Female,36,10,0.00,2,0,1,186180.42,0
3354,695,France,Female,36,2,0.00,2,0,1,167749.54,0
8915,606,France,Male,36,1,155655.46,1,1,1,192387.51,1
3752,717,Spain,Female,36,2,164557.95,1,0,1,82336.73,0


In [8]:
#Encode Gender using LabelEncoder
le=LabelEncoder()
df["Gender"]=le.fit_transform(df["Gender"])

In [9]:
df["Geography"].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [ ]:
###Using get_dummies
# df=pd.get_dummies(df,columns=["Geography"],prefix="Geography",dtype= int )

In [10]:
from sklearn.preprocessing import OneHotEncoder
ohe= OneHotEncoder()
geo_encoder=ohe.fit_transform(df[["Geography"]])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [11]:
ohe.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [12]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=ohe.get_feature_names_out(["Geography"]))

In [18]:
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [19]:
df=pd.concat([df.drop("Geography",axis=1),geo_encoded_df ],axis=1)
df.head()

KeyError: "['Geography'] not found in axis"

In [20]:
##Save encoders

with open("le.pkl","wb") as file:
    pickle.dump(le,file)

with open("ohe.pkl","wb") as file:
    pickle.dump(ohe,file)

In [21]:
#Divide dataset into independent and dependent features
X=df.drop('Exited',axis=1)

y=df['Exited']


In [22]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [23]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [24]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

<h2>ANN implementation</h2>

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [29]:
X_train.shape[1]

12

In [30]:
##Build our ANN model

model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #HL1 connected with i/p layer
    Dense(32,activation='relu'), #HL2    
    Dense(1,activation='sigmoid') #O/p layer

]

)

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [ ]:
##compile the model 
##Select Optimizers and loss function
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
##Set up the Tensorboard
from keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit/" + datetime.datetime.now().strftime( "%Y%m%d-%H%M%S")
tensorflow_callback =TensorBoard(log_dir=log_dir,histogram_freq=1)

In [49]:
##Set up EarlyStopping

early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights = True)

In [50]:
##Training the model
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
    
)

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.1777 - accuracy: 0.9170 - val_loss: 0.9246 - val_accuracy: 0.8165
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.1771 - accuracy: 0.9156 - val_loss: 0.9131 - val_accuracy: 0.8130
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.1711 - accuracy: 0.9183 - val_loss: 0.9309 - val_accuracy: 0.8110
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.1699 - accuracy: 0.9181 - val_loss: 0.9069 - val_accuracy: 0.8090
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.1651 - accuracy: 0.9212 - val_loss: 0.9676 - val_accuracy: 0.8055
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.1732 - accuracy: 0.9159 - val_loss: 0.8623 - val_accuracy: 0.8025
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.1638 - accuracy: 0.9200 - val_loss: 0.9244 - val_accuracy: 0.8175

In [51]:
model.save("model.h5")

c:\Users\Mayan\OneDrive\Desktop\ANN\myenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [62]:
##Load tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [65]:
%tensorboard --logdir logs\fit 

Reusing TensorBoard on port 6007 (pid 12044), started 0:08:14 ago. (Use '!kill 12044' to kill it.)

<h1>ANN prediction</h1>

In [70]:
y_pred = model.predict(X_test)
print(y_pred)

63/63 [==============================] - 0s 1ms/step
[[1.6531337e-14]
 [4.5526010e-09]
 [3.7772451e-03]
 ...
 [6.1422545e-01]
 [1.1653999e-02]
 [6.1422545e-01]]


In [76]:
y_pred_classes= (y_pred > 0.5).astype(int)

In [86]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

print(confusion_matrix(y_test,y_pred_classes))
print(classification_report(y_test,y_pred_classes))
print(accuracy_score(y_test,y_pred_classes))

[[1392  215]
 [ 180  213]]
              precision    recall  f1-score   support

           0       0.89      0.87      0.88      1607
           1       0.50      0.54      0.52       393

    accuracy                           0.80      2000
   macro avg       0.69      0.70      0.70      2000
weighted avg       0.81      0.80      0.81      2000

0.8025


In [90]:
ohe.categories_[0]

array(['France', 'Germany', 'Spain'], dtype=object)